(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Analyzing hotel ratings on Tripadvisor

In this homework we will focus on practicing two techniques: web scraping and regression. For the first part, we will build upon the sample code from the Lecture and attempt to get some basic information for each hotel. Then, we will fit a regression model on this information and try to analyze it.   

One of the main disadvantages of scraping a website instead of using an API is that, without any notice, the website may change its layout and render our code useless. Something like that happened in our case. Tripadvisor changed the layout of the buttons that we use to navigate between the different pages of the results. This was the main reason people were having problem with executing the code.

**Task 1 (20 pts)**

The first task of the homework is to fix the scraping code. We basically need to replace the part where we are checking if there is another page and getting its link with new code that reflects the new navigation layout. 

** Task 2 (30 pts)**

Then, for each hotel that our search returns, we will "click" (with the code of course) on it and scrape the information below.

![Information to be scraped](hotel_info.png)

Of course, feel free to collect even more data if you want. 

** Task 3 (20 pts) **

Now, we will use regression to analyze this information. First, we will fit a linear regression model that predicts the average rating. For example, for the hotel above, the average rating is

$$ \text{AVG_SCORE} = \frac{1*31 + 2*33 + 3*98 + 4*504 + 5*1861}{2527}$$

Use the model to analyze the important factors that decide the $\text{AVG_SCORE}$.

** Task 4 (30 pts) **

Finally, we will use logistic regression to decide if a hotel is _excellent_ or not. We classify a hotel as _excellent_ if more than **60%** of its ratings are 5 stars. This is a binary attribute on which we can fit a logistic regression model. As before, use the model to analyze the data.

-------

In order to use code from a Python script file, we need to put that file in the same folder as the notebook and import it as a library. Then, we will be able to access it's functions. For example, in the case of the lecture code, we could do the following:

``` python
import scrape_solution as scrape

scrape.get_city_page()
```

Of course, you might need to modify and restructure the code so that it returns what you need.

----

In [5]:
import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy as sp
import scipy.sparse.linalg as linalg
import scipy.cluster.hierarchy as hr
from scipy.spatial.distance import pdist, squareform

import sklearn.datasets as datasets
import sklearn.metrics as metrics
import sklearn.utils as utils
import sklearn.linear_model as linear_model
import sklearn.svm as svm
import sklearn.cross_validation as cross_validation
import sklearn.cluster as cluster
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

from patsy import dmatrices

import networkx as nx

import seaborn as sns
%matplotlib inline

In [6]:
import scrape_solution as scrape
#finds the hotels, number of stars, and number of ratings for the state and city of boston
scrape.scrape_hotels("boston", "massachusetts")
#returns a list of the urls
urls = scrape.getUrls()

[2015-04-06 18:26:54,627] #################################### Option 2 ######################################
INFO:scrape_solution:#################################### Option 2 ######################################
[2015-04-06 18:26:54,716] #################################### Option 3 ######################################
INFO:scrape_solution:#################################### Option 3 ######################################
[2015-04-06 18:26:54,811] Hotel name: Hyatt Boston Harbor
INFO:scrape_solution:Hotel name: Hyatt Boston Harbor
[2015-04-06 18:26:54,812] Stars: 4
INFO:scrape_solution:Stars: 4
[2015-04-06 18:26:54,813] Number of reviews: 1,171 reviews 
INFO:scrape_solution:Number of reviews: 1,171 reviews 
[2015-04-06 18:26:54,815] Hotel name: Seaport Boston Hotel
INFO:scrape_solution:Hotel name: Seaport Boston Hotel
[2015-04-06 18:26:54,815] Stars: 4.5
INFO:scrape_solution:Stars: 4.5
[2015-04-06 18:26:54,818] Number of reviews: 2,551 reviews 
INFO:scrape_solution:Number of re

SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [10]:
#goes through all the hotels pages and appends the information to a global dictionary
scrape.parse_hotelpages(urls)
#returns the dictionary that contains everything I need
hotel_dt = scrape.getDict()
#converts it to a dataframe. 
hotel_db =pd.DataFrame(hotel_dt)
hotel_db
hotel_db = hotel_db.T

Americas Best Value Inn  Ames Boston Hotel  \
Average                                  8.0               89.0   
Business                                 3.0              153.0   
Cleanliness                              3.0                4.5   
Couples                                  3.0              431.0   
Excellent                                2.0              473.0   
Families                                 4.0              103.0   
Location                                 3.0                5.0   
Poor                                     1.0               35.0   
Rooms                                    2.5                4.5   
Service                                  2.5                4.5   
Sleep Quality                            2.5                4.5   
Solo                                     2.0               65.0   
Terrible                                 5.0               15.0   
Value                                    3.0                4.0   
Very good                                2.0              258.0   
total number rating                     18.0              870.0   
total rating                             2.5                4.5   

                     BEST WESTERN PLUS Roundhouse Suites  \
Average                                            129.0   
Business                                           104.0   
Cleanliness                                          4.0   
Couples                                            179.0   
Excellent                                          179.0   
Families                                           257.0   
Location                                             4.0   
Poor                                                69.0   
Rooms                                                4.0   
Service                                              4.0   
Sleep Quality                                        3.5   
Solo                                                17.0   
Terrible                                            34.0   
Value                                                4.0   
Very good                                          305.0   
total number rating                                716.0   
total rating                                         3.5   

                     BEST WESTERN University Hotel Boston-Brighton  \
Average                                                       78.0   
Business                                                      49.0   
Cleanliness                                                    4.0   
Couples                                                       59.0   
Excellent                                                     97.0   
Families                                                     166.0   
Location                                                       4.0   
Poor                                                          33.0   
Rooms                                                          3.5   
Service                                                        4.0   
Sleep Quality                                                  4.0   
Solo                                                          27.0   
Terrible                                                      21.0   
Value                                                          4.0   
Very good                                                    132.0   
total number rating                                          361.0   
total rating                                                   3.5   

                     Battery Wharf Hotel, Boston Waterfront  \
Average                                                74.0   
Business                                              128.0   
Cleanliness                                             4.5   
Couples                                               407.0   
Excellent                                             560.0   
Families                                              175.0   
Location                       

In [30]:

avg_ls = []

for idx, row in hotel_db.iterrows():
    average = (5*row["Excellent"]) + (4*row["Very good"]) + (3*row["Average"]) + (2*row["Poor"]) + row["Terrible"]
    avg_rate = average /(row["total number rating"])
    avg_ls.append(avg_rate)
hotel_db["Average rating"] = avg_ls
hotel_db
# avg_ls

Average  Business  \
Americas Best Value Inn                                       8         3   
Ames Boston Hotel                                            89       153   
BEST WESTERN PLUS Roundhouse Suites                         129       104   
BEST WESTERN University Hotel Boston-Brighton                78        49   
Battery Wharf Hotel, Boston Waterfront                       74       128   
Beacon Hill Hotel and Bistro                                 18        13   
Boston Harbor Hotel                                          44       255   
Boston Hotel Buckminster                                    149       106   
Boston Marriott Copley Place                                332       710   
Boston Marriott Long Wharf                                  137       253   
Boston Park Plaza                                          1013      1190   
Chandler Inn                                                110       120   
Charlesmark Hotel                                            71       140   
Club Quarters in Boston                                     133       239   
Colonnade Hotel                                             160       526   
Comfort Inn - Boston                                         85        41   
Constitution Inn                                            115        42   
Copley House                                                 60        67   
Copley Square Hotel                                         103       223   
Courtyard Boston Downtown                                   266       224   
Courtyard Boston-South Boston                                56        73   
Courtyard by Marriott Boston Copley Square                   43       300   
Courtyard by Marriott Boston Logan Airport                  102       138   
Days Hotel Boston                                            57        41   
DoubleTree Club by Hilton Hotel Boston Bayside              228       224   
DoubleTree Suites by Hilton Boston-Cambridge                138       293   
DoubleTree by Hilton Hotel Boston - Downtown                 61       145   
Eliot Hotel                                                  65       117   
Embassy Suites Boston Logan Airport                         217       302   
Four Seasons Hotel Boston                                    47       317   
...                                                         ...       ...   
Milner Hotel                                                112        56   
Newbury Guest House                                          44        45   
Nine Zero Hotel - a Kimpton Hotel                           105       324   
Omni Parker House                                           563      1014   
Onyx Hotel - a Kimpton Hotel                                109       148   
Ramada Boston                                               256       163   
Renaissance Boston Waterfront Hotel                         104       371   
Residence Inn Boston Back Bay / Fenway                       23        86   
Residence Inn Boston Downtown Seaport                         6        53   
Residence Inn by Marriott Boston Harbor on Tudor Wharf       42        71   
Revere Hotel Boston Common                                  181       300   
Seaport Boston Hotel                                         98      1113   
Sheraton Boston Hotel                                        74       203   
Taj Boston                                                  149       244   
The Boston Common Hotel and Conference Center               147        80   
The Boxer Boston                                             45        56   
The Fairmont Copley Plaza, Boston                           157       444   
The Inn At St Botolph                                        20        87   
The Inn at Longwood Medical                                  96       187   
The Langham, Boston                                         112       324   
The Liberty Hotel                                   

In [2]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()